## Подготовка

In [8]:
# Предварительная настройка системы
import numpy as np
import pandas as pd

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)

import datetime
from datetime import datetime, date

## Загрузка

In [10]:
# Настройки загрузки
dir_pickle = '../pickle/'
data_pickle = 'data.pkl'
history_pickle = 'history.pkl'
history_statuses_piclke = 'history_statuses.pkl'
sprints_pickle = 'sprints.pkl'
sprints_tasks_pickle = 'sprints_tasks.pkl'

In [11]:
# Загружаем
dD = pd.read_pickle(f"{dir_pickle}{data_pickle}")
dS = pd.read_pickle(f"{dir_pickle}{sprints_pickle}")
dSE = pd.read_pickle(f"{dir_pickle}{sprints_tasks_pickle}")
dH = pd.read_pickle(f"{dir_pickle}{history_pickle}")
dHS = pd.read_pickle(f"{dir_pickle}{history_statuses_piclke}")
# TODO Неаккуратно настроено, переписать

## Срез по спринту

In [13]:
# Выбираем спринт
current_sprint = dS.iloc[4]

In [14]:
# Все изменения статусов в выбранном спринте и раньше
status_changes_on_sprint = dHS.loc[dHS['history_date'] < current_sprint['sprint_end_date']]

In [15]:
# Группируем по id задачи
changes_grouped_by_task = status_changes_on_sprint.groupby('entity_id')
# changes_grouped_by_task.size()

In [16]:
# Находим последние записи
last_changes = changes_grouped_by_task.apply(lambda x: x.nlargest(1, 'history_version'), include_groups=False)
# last_changes

In [17]:
# Выявляем актуальные для спринта
actual_for_sprint = last_changes.loc[~((last_changes['after'] == 'closed') & (last_changes['history_date'] < current_sprint['sprint_start_date']))]
# actual_for_sprint

## К выполнению

In [61]:
# Список задач
todo_list = list(map(lambda x: x[0], list(created_in_sprint.index)))

In [69]:
# Статус категории «Создано»
created_in_sprint = actual_for_sprint.loc[actual_for_sprint['after'] == 'created']
# created_in_sprint

In [73]:
# Срез для вычисления метрики
todo_tasks = dD.loc[dD.index.isin(todo_list)]

In [142]:
metrics_todo = round(todo_tasks['estimation'].sum() / 3600, 1)
metrics_todo

68.0

## В работе

In [116]:
# Срез для вычисления метрики
inwork_in_sprint = actual_for_sprint.loc[(actual_for_sprint['after'] != 'done') & (actual_for_sprint['after'] != 'rejectedByThePerformer')]
# inwork_in_sprint['after'].unique()

In [122]:
# Список задач
inwork_list = list(map(lambda x: x[0], list(inwork_in_sprint.index)))
# inwork_list

In [146]:
# Задачи в работе
inwork_tasks = dD.loc[dD.index.isin(inwork_list)]
# inwork_tasks

In [150]:
metrics_inwork = round(inwork_tasks['estimation'].sum() / 3600, 1)
metrics_inwork

6578.1